In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from main import *

from sklearn.cluster import KMeans
from datasetUtils import load_from_Jadson
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)


# if __name__ == '__main__':
# parser = argparse.ArgumentParser(description='Define the UDA parameters')
#
# parser.add_argument('--gpu_ids', type=str, default="7", help='GPU IDs')
# parser.add_argument('--lr', type=float, default=3.5e-4, help='Learning Rate')
# parser.add_argument('--P', type=int, default=16, help='Number of Persons')
# parser.add_argument('--K', type=int, default=4, help='Number of samples per person')
# parser.add_argument('--tau', type=float, default=0.05, help='tau value used on softmax triplet loss')
# parser.add_argument('--beta', type=float, default=0.999, help='beta used on self-Ensembling')
# parser.add_argument('--k1', type=int, default=30, help='k on k-Reciprocal Encoding')
# parser.add_argument('--sampling', type=str, default="mean", help='Mean or Random feature vectors to be prototype')
# parser.add_argument('--lambda_hard', type=float, default=0.5, help='tuning prameter of Softmax Triplet Loss')
# parser.add_argument('--num_iter', type=int, default=400, help='Number of iterations on an epoch')
# parser.add_argument('--momentum_on_feature_extraction', type=int, default=0,
# help='If it is the momentum used on feature extraction')
# parser.add_argument('--target', type=str, help='Name of target dataset')
# parser.add_argument('--path_to_save_models', type=str, help='Path to save models')
# parser.add_argument('--path_to_save_metrics', type=str, help='Path to save metrics (mAP, CMC, ...)')
# parser.add_argument('--version', type=str, help='Path to save models')
# parser.add_argument('--eval_freq', type=int, help='Evaluation Frequency along training')

# args = parser.parse_args()
# gpu_ids = args.gpu_ids
# base_lr = args.lr
# P = args.P
# K = args.K

# tau = args.tau
# beta = args.beta
# k1 = args.k1
# sampling  = args.sampling
#
# lambda_hard = args.lambda_hard
# number_of_iterations = args.num_iter
# momentum_on_feature_extraction = bool(args.momentum_on_feature_extraction)
# target = args.target
# dir_to_save = args.path_to_save_models
# dir_to_save_metrics = args.path_to_save_metrics
# version = args.version
# eval_freq = args.eval_freq
# main.py --gpu_ids=0,1,2,3 --lr=3.5e-4 --P=16 --K=12 --tau=0.04 --beta=0.999 --k1=30 --sampling=mean --lambda_hard=0.5 --num_iter=7 --momentum_on_feature_extraction=0 --target=Duke --path_to_save_models=models --path_to_save_metrics=metrics --version=version_name --eval_freq=5

import sys
import os
import pandas as pd

from IPython.display import display, Image

from IPython.display import display, HTML
from bs4 import BeautifulSoup


# Função para exibir a imagem usando HTML
def exibir_imagem(imagem_path):
    return f'<img src="{imagem_path}" width="400">'


from metricas import *

html_content= ""
df = pd.DataFrame({
    'k':[], 
    'lambda_hard':[],
    'modelo':[],
    'matriz_confusao':[], 
    'Acuracia':[], 
    'Precisao':[],
    'Recall':[],
    'F1-score':[],
    'Grafico':[],
    'Tipo':[]
    })

gpus = "0,1,2,3" 
for k in [4]:
    for lambda_hard in [ 0.0 ]:
                
        print(f"**** inicio do teste com ruido em k:{k} e lambda_hard:{lambda_hard} ****")
        
        version = f"teste-03-ruido_{k}_{lambda_hard}"
        
        main(gpu_ids=gpus,base_lr=3.5e-4,P=16,K=k,tau=0.04,beta=0.999,k1=30,sampling="random",lambda_hard=lambda_hard,number_of_iterations=7,momentum_on_feature_extraction=0,target="Jadson",dir_to_save="models",dir_to_save_metrics="metrics",version=version,eval_freq=5,use_ruido=True)
        
        for metodo in ["resnet50","osnet","densenet121","mean"]:
            metricas_t, metricas_v, rotulos_t, rotulos_v = metricas(k=k, lambda_hard=lambda_hard, modelo=metodo)
        
            for count in range( 0, metricas_t.shape[0] ):
                linha = {
                'k':[k], 
                'lambda_hard':[lambda_hard],
                'modelo':[metodo],
                'matriz_confusao':[f'resultados/MC_{k}_{lambda_hard}_{count}_{metodo}_test.png'], 
                rotulos_t[0]: [metricas_t[count][0]], 
                rotulos_t[1]: [metricas_t[count][1]], 
                rotulos_t[2]: [metricas_t[count][2]],
                rotulos_t[3]: [metricas_t[count][3]],
                rotulos_t[4]: [metricas_t[count][4]],
                rotulos_t[5]: [metricas_t[count][5]],
                rotulos_t[6]: [metricas_t[count][6]],
                rotulos_t[7]: [metricas_t[count][7]], 
                'Grafico':  [f'resultados/grafico_{k}_{lambda_hard}_{count}_{metodo}_test.png'],
                'Tipo':     'Test'
                    }
                df = pd.concat( [df, pd.DataFrame(linha)], axis=0)
            
            for count in range( 0, metricas_v.shape[0] ):
               linha = {
               'k':[k], 
               'lambda_hard':[lambda_hard],
               'modelo':[metodo],
               'matriz_confusao':[f'resultados/MC_{k}_{lambda_hard}_{count}_{metodo}_valid.png'], 
               rotulos_v[0]: [metricas_v[count][0]], 
               rotulos_v[1]: [metricas_v[count][1]], 
               rotulos_v[2]: [metricas_v[count][2]], 
               rotulos_v[3]: [metricas_v[count][3]], 
               rotulos_v[4]: [metricas_v[count][4]], 
               rotulos_v[5]: [metricas_v[count][5]], 
               rotulos_v[6]: [metricas_v[count][6]], 
               rotulos_v[7]: [metricas_v[count][7]], 
               'Grafico':  [f'resultados/grafico_{k}_{lambda_hard}_{count}_{metodo}_valid.png'],
               'Tipo':     'Valid'
                   }
               df = pd.concat( [df, pd.DataFrame(linha)], axis=0)
        
        # Aplicar a função à coluna 'imagem' e criar uma nova coluna 'imagem_exibicao'
        df['MC'] = df['matriz_confusao'].apply(exibir_imagem)
        df['GR'] = df['Grafico'].apply(exibir_imagem)
        
        html_content = df[['k', 
                           'lambda_hard', 
                           'Tipo', 
                           'modelo', 
                           rotulos_v[0], 
                           rotulos_v[1], 
                           rotulos_v[2], 
                           rotulos_v[3], 
                           rotulos_v[4], 
                           rotulos_v[5], 
                           rotulos_v[6], 
                           rotulos_v[7], 
                           'MC', 
                           'GR']].to_html(escape=False, index=False)
        # salvando df em arquivo html
        # Use BeautifulSoup para formatar o HTML
        soup = BeautifulSoup(html_content, 'html.parser')
        formatted_html = soup.prettify()
        
        # Salve o HTML em um arquivo
        head = "<!DOCTYPE html>\n<html lang='pt-br'>\n<head>\n  <meta charset='UTF-8'>\n  <meta name='viewport' content='width=device-width, initial-scale=1.0'>\n  <style>\n    table {\n      width: 100%;\n      border-collapse: collapse;\n    }\n    th, td {\n      border: 1px solid #ddd;\n      padding: 8px;\n      text-align: left;\n    }\n    th {\n      background-color: #f2f2f2;\n    }\n    thead th {\n      position: sticky;\n      top: 0;\n      z-index: 1;\n      background-color: #f2f2f2;    }\n  </style>\n    <title>Relatório Parcial</title>\n</head>\n<body>"
        with open('relatorio-17-3.html', 'w', encoding='utf-8') as file:
            file.write(head)
            file.write(formatted_html)
            file.write('</body></html>')



/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/torchreid/reid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly

**** inicio do teste com ruido em k:4 e lambda_hard:0.0 ****
Num GPU's: 3
Allocated GPU's for model: [1, 2]


/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Successfully loaded imagenet pretrained weights from "/home/emorais/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
Successfully loaded imagenet pretrained weights from "/home/emorais/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"


/home/emorais/miniconda3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


...

Jaccard distance computing time cost: 71.7138102054596
Computing jaccard distance...


bruteForceKnn is deprecated; call bfKnn instead


Jaccard distance computing time cost: 70.88718056678772
Computing jaccard distance...


bruteForceKnn is deprecated; call bfKnn instead


Jaccard distance computing time cost: 78.26744508743286
Reliability: 0.998
Mean Purity: 0.03188
There are 3 clusters with 4 cameras
There are 3 clusters with 5 cameras
There are 1 clusters with 6 cameras
There are 2 clusters with 7 cameras
There are 2 clusters with 8 cameras
There are 2 clusters with 9 cameras
There are 2 clusters with 11 cameras
There are 1 clusters with 12 cameras
There are 1 clusters with 15 cameras
There are 1 clusters with 17 cameras
There are 1 clusters with 19 cameras
There are 1 clusters with 31 cameras
There are 1 clusters with 33 cameras
There are 1 clusters with 45 cameras
There are 1 clusters with 46 cameras
There are 3 clusters with 52 cameras
There are 1 clusters with 53 cameras
There are 1 clusters with 54 cameras
There are 1 clusters with 55 cameras
There are 2 clusters with 56 cameras
There are 2 clusters with 57 cameras
There are 1 clusters with 58 cameras
There are 4 clusters with 59 cameras
There are 3 clusters with 60 cameras
There are 12 clusters 

In [2]:
# Exibir o DataFrame com as imagens
display(HTML(html_content))
print(df)

k,lambda_hard,Tipo,modelo,ACCURACY,PRECISION,RECALL,F1_SCORE,EER,HTER,APCER,BPCER,MC,GR
4.0,0.0,Test,resnet50,0.800000,0.800175,0.999635,0.888853,-0.01,-0.01,-0.01,-0.01,,
4.0,0.0,Valid,resnet50,0.786667,0.797297,0.983333,0.880597,-0.01,-0.01,-0.01,-0.01,,
4.0,0.0,Test,osnet,0.772750,0.810069,0.935208,0.868153,-0.01,-0.01,-0.01,-0.01,,
4.0,0.0,Valid,osnet,0.786667,0.797297,0.983333,0.880597,-0.01,-0.01,-0.01,-0.01,,
4.0,0.0,Test,densenet121,0.802417,0.802469,0.998906,0.889977,-0.01,-0.01,-0.01,-0.01,,
4.0,0.0,Valid,densenet121,0.786667,0.797297,0.983333,0.880597,-0.01,-0.01,-0.01,-0.01,,
4.0,0.0,Test,mean,0.795750,0.802207,0.988385,0.885617,-0.01,-0.01,-0.01,-0.01,,
4.0,0.0,Valid,mean,0.786667,0.797297,0.983333,0.880597,-0.01,-0.01,-0.01,-0.01,,


     k  lambda_hard       modelo                              matriz_confusao  \
0  4.0          0.0     resnet50      resultados/MC_4_0.0_0_resnet50_test.png   
0  4.0          0.0     resnet50     resultados/MC_4_0.0_0_resnet50_valid.png   
0  4.0          0.0        osnet         resultados/MC_4_0.0_0_osnet_test.png   
0  4.0          0.0        osnet        resultados/MC_4_0.0_0_osnet_valid.png   
0  4.0          0.0  densenet121   resultados/MC_4_0.0_0_densenet121_test.png   
0  4.0          0.0  densenet121  resultados/MC_4_0.0_0_densenet121_valid.png   
0  4.0          0.0         mean          resultados/MC_4_0.0_0_mean_test.png   
0  4.0          0.0         mean         resultados/MC_4_0.0_0_mean_valid.png   

   Acuracia  Precisao  Recall  F1-score  \
0       NaN       NaN     NaN       NaN   
0       NaN       NaN     NaN       NaN   
0       NaN       NaN     NaN       NaN   
0       NaN       NaN     NaN       NaN   
0       NaN       NaN     NaN       NaN   
0       NaN 